In [5]:
from typing import List


class Iris:

    def __init__(self, points: List[float], category: str):
        self.points = points
        self.category = category

    def __repr__(self):
        return self.__str__()

    def __str__(self):
        return f"(Iris: {self.points}, kategoria: {self.category})"

    def __eq__(self, other):
        for i in range(len(self.points)):
            if self.points[i] != other.points[i]:
                return False
        return self.category == other.category

    def __hash__(self) -> int:
        hash = sum([point.__hash__() for point in self.points])
        hash += self.category.__hash__()
        return hash


In [6]:
categories = set()
trainingLeafs = set()

with open('training_set.txt', 'r') as f:
    data = f.read()
    data = data.splitlines()
    for line in data:
        lineValues = line.split(',')
        category = lineValues.pop()
        value = [float(x) for x in lineValues]
        categories.add(category)
        trainingLeafs.add(Iris(value, category))

print(categories)
print(trainingLeafs)

expectedLeafs = set()
with open('test_set.txt', 'r') as f:
    data = f.read()
    data = data.splitlines()
    for line in data:
        lineValues = line.split(',')
        category = lineValues.pop()
        value = [float(x) for x in lineValues]
        expectedLeafs.add(Iris(value, category))

{'Iris-versicolor', 'Iris-virginica'}
{(Iris: [5.6, 2.5, 3.9, 1.1], kategoria: Iris-versicolor), (Iris: [5.0, 2.0, 3.5, 1.0], kategoria: Iris-versicolor), (Iris: [6.7, 3.0, 5.0, 1.7], kategoria: Iris-versicolor), (Iris: [5.6, 3.0, 4.5, 1.5], kategoria: Iris-versicolor), (Iris: [6.1, 2.6, 5.6, 1.4], kategoria: Iris-virginica), (Iris: [6.1, 2.9, 4.7, 1.4], kategoria: Iris-versicolor), (Iris: [6.7, 3.1, 4.4, 1.4], kategoria: Iris-versicolor), (Iris: [6.2, 2.2, 4.5, 1.5], kategoria: Iris-versicolor), (Iris: [6.3, 2.9, 5.6, 1.8], kategoria: Iris-virginica), (Iris: [5.8, 2.7, 3.9, 1.2], kategoria: Iris-versicolor), (Iris: [5.5, 2.3, 4.0, 1.3], kategoria: Iris-versicolor), (Iris: [6.3, 3.3, 6.0, 2.5], kategoria: Iris-virginica), (Iris: [6.4, 2.7, 5.3, 1.9], kategoria: Iris-virginica), (Iris: [5.5, 2.4, 3.7, 1.0], kategoria: Iris-versicolor), (Iris: [6.2, 2.8, 4.8, 1.8], kategoria: Iris-virginica), (Iris: [6.8, 2.8, 4.8, 1.4], kategoria: Iris-versicolor), (Iris: [6.9, 3.1, 4.9, 1.5], kategoria

In [7]:
from copy import deepcopy
import random
from typing import List


def dot(input_data: List[float], weights: List[float]):
    return sum([x[0] * x[1] for x in zip(input_data, weights)])


class Perceptron:
    def __init__(self, learning_rate):
        self.weights = []
        self.threshold = random.random()
        self.learning_rate = learning_rate

    def set_weights(self, weights: List[float]):
        self.weights = weights

    def get_result(self, input_data: List[float]):
        if len(self.weights) == 0:
            self.weights = [random.random() for x in range(len(input_data))]
        elif len(input_data) != len(self.weights):
            raise Exception("Wrong input data for this perceptron!")
        return int(dot(input_data, weights=self.weights) >= self.threshold)

    def learn(self, input_data: List[float], expected: int):
        given = self.get_result(input_data)
        # print(f"Given: {given}")
        self.weights.append(self.threshold)
        # print(f"Weight before: {self.weights}")
        input_data.append(-1)
        # print(f"Input before: {input_data}")
        diff = [(expected - given) * self.learning_rate * x for x in input_data]
        # print(f"Diff: {diff}")
        input_data.pop()
        # print(f"Input after: {input_data}")
        self.weights = [x[0] + x[1] for x in zip(self.weights, diff)]
        # print(f"Weights after: {self.weights}")
        self.threshold = self.weights.pop()
        # print(f"New threshold: {self.threshold}")


translation = {'Iris-virginica': 0, 'Iris-versicolor': 1}
inv_translation = dict((v, k) for k, v in translation.items())
learning_rate = 0.1

iris_detailed_accuracy = {'Iris-virginica': 0.0, 'Iris-versicolor': 0.0}
next_iris_detailed_accuracy = {'Iris-virginica': 0.0, 'Iris-versicolor': 0.0}
accurancy = 0.0
next_accurancy = 0.0

perceptron = Perceptron(0.1)
next_perceptron = deepcopy(perceptron)

while accurancy <= next_accurancy:
    accurancy = next_accurancy
    next_accurancy = 0.0
    iris_detailed_accuracy = deepcopy(next_iris_detailed_accuracy)
    next_iris_detailed_accuracy = {'Iris-virginica': 0.0, 'Iris-versicolor': 0.0}
    for leaf in trainingLeafs:
        next_perceptron.learn(leaf.points, translation[leaf.category])
    for leaf in expectedLeafs:
        result = next_perceptron.get_result(leaf.points)
        check = inv_translation[result]
        if leaf.category == check:
            next_accurancy += 1
            next_iris_detailed_accuracy[leaf.category] += 1
    next_accurancy /= len(expectedLeafs)
    for key in iris_detailed_accuracy:
        next_iris_detailed_accuracy[key] /= len(list(filter(lambda x: x.category == key, expectedLeafs)))
    if (next_accurancy > accurancy):
        perceptron = deepcopy(next_perceptron)
    print(accurancy*100)

print(f"Poprawność: {accurancy*100}%")
print(f"Poprawność dla Iris-virginica: {iris_detailed_accuracy['Iris-virginica']*100}%")
print(f"Poprawność dla Iris-versicolor: {iris_detailed_accuracy['Iris-versicolor']*100}%")


0.0
56.666666666666664
100.0
Poprawność: 100.0%
Poprawność dla Iris-virginica: 100.0%
Poprawność dla Iris-versicolor: 100.0%


In [10]:
inputValues = input("Podaj 4 liczby: ")
inputFloats = [float(x) for x in inputValues.split(',')]
print(f"Podane wartości: {inputFloats}")

leaf = Iris(inputFloats, category=inv_translation[perceptron.get_result(leaf.points)])

print(f"Model przewiduje, że kwiat jest gatunku: {leaf.category}")

Podane wartości: [1.1, 2.2, 3.3, 4.4]
Model przewiduje, że kwiat jest gatunku: Iris-virginica
